In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
CONNECTION_STRING = os.getenv("CONNECTION_STRING")

# Create SQLAlchemy engine
engine = create_engine(CONNECTION_STRING)

print("✅ Connected to Neon database")

✅ Connected to Neon database


In [2]:
# Check total rows
total_rows = pd.read_sql("SELECT COUNT(*) AS total_rows FROM sales_data;", engine)
print(total_rows)

# Preview first 5 rows
df_preview = pd.read_sql("SELECT * FROM sales_data LIMIT 5;", engine)
df_preview

   total_rows
0      392692


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,InvoiceMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010-12


In [3]:
query = "SELECT * FROM sales_data LIMIT 5;"
df_check = pd.read_sql(query, engine)
print(df_check.columns)

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'TotalPrice', 'InvoiceMonth'],
      dtype='object')


# Total Revenue

In [4]:
query_total_revenue = """
SELECT ROUND(CAST(SUM("TotalPrice") AS numeric), 2) AS total_revenue
FROM sales_data;
"""
total_revenue = pd.read_sql(query_total_revenue, engine)
print(total_revenue)

   total_revenue
0     8887208.89


# top 10 countries

In [5]:
query_sales_by_country = """
SELECT "Country", 
       ROUND(CAST(SUM("TotalPrice") AS numeric), 2) AS total_revenue
FROM sales_data
GROUP BY "Country"
ORDER BY total_revenue DESC;
"""

sales_by_country = pd.read_sql(query_sales_by_country, engine)
display(sales_by_country.head(10))  # top 10 countries

,Country,total_revenue
0,United Kingdom,7285024.64
1,Netherlands,285446.34
2,EIRE,265262.46
3,Germany,228678.40
4,France,208934.31
5,Australia,138453.81
6,Spain,61558.56
7,Switzerland,56443.95
8,Belgium,41196.34
9,Sweden,38367.83


# Top 10 customers by total spend.

In [6]:
query_top_customers = """
SELECT "CustomerID",
       ROUND(CAST(SUM("TotalPrice") AS numeric), 2) AS total_spent
FROM sales_data
WHERE "CustomerID" IS NOT NULL
GROUP BY "CustomerID"
ORDER BY total_spent DESC
LIMIT 5;
"""

top_customers = pd.read_sql(query_top_customers, engine)
display(top_customers)

,CustomerID,total_spent
0,14646,280206.02
1,18102,259657.30
2,17450,194390.79
3,16446,168472.50
4,14911,143711.17


# Monthly Sales Trend

In [7]:
query_monthly_revenue = """
SELECT "InvoiceMonth",
       ROUND(CAST(SUM("TotalPrice") AS numeric), 2) AS monthly_revenue
FROM sales_data
GROUP BY "InvoiceMonth"
ORDER BY "InvoiceMonth";
"""

monthly_revenue = pd.read_sql(query_monthly_revenue, engine)
display(monthly_revenue)

,InvoiceMonth,monthly_revenue
0,2010-12,570422.73
1,2011-01,568101.31
2,2011-02,446084.92
3,2011-03,594081.76
4,2011-04,468374.33
5,2011-05,677355.15
6,2011-06,660046.05
7,2011-07,598962.90
8,2011-08,644051.04
9,2011-09,950690.20


import plotly.express as px

fig = px.line(monthly_revenue, 
              x="InvoiceMonth", 
              y="monthly_revenue",
              title="Monthly Revenue Trend",
              markers=True)
fig.show()